### **imports**

In [83]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import upsilon

### **making a table of all of the stars in the txt file**

In [84]:
column1 = []
column2 = []
column3 = []
column4 = []

with open('rrlyr_vsx_clean.txt', 'r') as file:
    for line in file:
        empty = []
        components = line.split(' ', 3)
    
        column1.append(components[0])
        column2.append(float(components[1]))
        column3.append(float(components[2]))
        column4.append(components[3].strip())

# column2 = column2 * u.deg
# column3 = column3 * u.deg

final_table = QTable([column1, column2, column3, column4], names = ("Star Type", "RA", "Dec", "Other Info"))

In [85]:
final_table

Star Type,RA,Dec,Other Info
str24,float64,float64,str30
RRAB,0.01307,-2.43007,PS1-3PI J000003.12-022548.2
RRAB,0.01521,35.36286,GM And
RRAB,0.01669,18.40698,CSS_J000004.0+182425
RRAB,0.02771,56.65331,V1023 Cas
RRAB,0.03171,34.67397,SERIV 27
RRAB,0.03901,5.58986,CSS_J000009.3+053523
RRAB,0.04274,-2.51629,PS1-3PI J000010.25-023058.6
RRAB,0.04331,-21.92086,CSS 120828:000010-215515
RRAB,0.04782,62.71524,Gaia DR2 430046902041301376


### **importing the asassn client**

In [86]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a SPECFIED index**

In [87]:
search = client.cone_search(ra_deg = final_table["RA"][4], dec_deg = final_table["Dec"][4], radius=0.0003, catalog='master_list', download=True)
print(search)

path = search.save(save_dir='/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/', file_format="csv")[1]

Pulled 1 of 1
LightCurveCollection with 1 light curves 
     asas_sn_id    ra_deg    dec_deg                catalog_sources
0  128850052915  0.031701  34.673978  [stellar_main, tic, aavsovsx]


### **making the csv file I just downloaded into a pandas dataframe**

In [88]:
new_file = pd.read_csv(path, skiprows=1)
new_file

,asas_sn_id,jd,flux,flux_err,mag,mag_err,limit,fwhm,image_id,camera,quality,phot_filter
0,128850052915,2.455973e+06,1.787761,0.049683,15.829008,0.030207,17.971844,2.38,bb006236,bb,G,V
1,128850052915,2.456221e+06,1.139000,0.046114,16.318472,0.044007,18.052766,2.04,bb020303,bb,G,V
2,128850052915,2.456224e+06,1.030333,0.097677,16.427338,0.103045,17.237873,1.77,bb020892,bb,B,V
3,128850052915,2.456234e+06,1.504700,0.056758,16.016157,0.041000,17.827294,1.76,bb023747,bb,G,V
4,128850052915,2.456236e+06,1.384540,0.050223,16.106518,0.039428,17.960107,2.00,bb023235,bb,G,V
...,...,...,...,...,...,...,...,...,...,...,...,...
901,128850052915,2.460272e+06,1.677626,0.069389,15.838328,0.044958,17.549409,1.55,br581373,br,G,g
902,128850052915,2.460288e+06,1.085520,0.039568,16.310971,0.039620,18.159280,1.70,br587130,br,G,g
903,128850052915,2.460295e+06,0.869506,0.037747,16.551884,0.047187,18.210424,1.61,br589414,br,G,g
904,128850052915,2.460297e+06,3.415947,0.068637,15.066288,0.021840,17.561246,1.48,br590367,br,G,g


### **filter that csv file for only g filters and save the output as a new txt file that can be run in upsilon**

In [89]:
g_filter = new_file[new_file['phot_filter'] == 'g']
web_format = g_filter[["jd", "mag", "mag_err"]]
web_format.to_csv('/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/128850052915.txt', sep='\t', index=False, header = False)

In [90]:
import upsilon

# Load a classification model.
# (takes time, do only once if processing multiple lcs)
rf_model = upsilon.load_rf_model()


############# Read input lc data #############
# Read the input lightcurve file
f = open("/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/128850052915.txt", 'r')
data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
f.close()

# Split it into three arrays
date=data['JD']
mag=data['mag']
err=data['err']

# Sigma-clip the input lightcurve
date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

############# Extract lightcurve features #############
e_features = upsilon.ExtractFeatures(date, mag, err)
e_features.run()
features = e_features.get_features()

############# Classify #############
# Classify the light curve
label, probability, flag = upsilon.predict(rf_model, features)

# Print results

print("\n\n######### Classification results #########\n")
print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
print("class = " + str(label) )
print("class_probability = " + str(probability) )
print("\n\n######### Lightcurve features #########\n")

for feature in features:
  print( str(feature) + " = " + str(features[feature]) )
#  print feature," = ",features[feature]

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.47169546291458164
cusum = 0.309062937555019
eta = 1.8669104190563452
hl_amp_ratio = 0.3124018295480004
kurtosis = -0.4193325941458528
n_points = 543
period = 0.46197284192571736
period_SNR = 125.8601819428668
period_log10FAP = -65.98128593791847
period_uncertainty = 9.267258987280536e-05
phase_cusum = 0.32929803539105806
phase_eta = 0.4682973095720427
phi21 = 0.518777939233599
phi31 = 1.3643490162560412
quartile31 = 0.6198908129808256
r21 = 0.42847678584203114
r31 = 0.2842978299081692
shapiro_w = 0.9052363038063049
skewness = -0.8032336837892786
slope_per10 = -0.026789237936296863
slope_per90 = 0.033965564312094486
stetson_k = 0.6232290966226436
weighted_mean = 15.890647032534282
weighted_std = 0.44133324832001314


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
